## Cost Function jupyter notebook

In [1]:
import pandas as pd
import numpy as np
import math 
from decimal import ROUND_DOWN, ROUND_UP


**User created Modules**


<pre>
.
&#8866; data
&#8866; input_data
&#8985; ..
</pre>

In [2]:
import data
import input_data

In [3]:
lamination = data.lamination_data
swg = data.swg_data

In [4]:
Frequency = 47 # Hz
Temperature_rise_goal = 100 # degree Celcius
Output_power = 250 # watts
Efficiency = 95 # %
Input_voltage = 115 # volts
Ouptut_voltage = 115 # volts
Regulation = 5 # in a scale of 100
Bobbin_thickness = 1.5 
K_f = 4.44
K_u = 0.4
B_ac = 1.6
J = 250
insulation_thickness = 0.2 #mm

pi = np.pi #pi
a = 1.68 # coefficients for core loss
b = 1.86 # coefficients for core loss 

Rate_of_Cu = 950 # Rs / Kg
Rate_of_Fe = 250 # Rs / Kg

Resistivity_Cu =  1.68 * 10**-6 # ohm cm

In [5]:
def area_product(K_f, K_u, B_ac, J, Frequency, apparent_power):
    """Input Arguments:
        Output power watts
        Efficiency in scale of 100%
        K_f
        K_u
        B_ac -> Operational flux density
        J
        Frequency -> in Hz
        Apparent power in watts
    """
    Area_product = (apparent_power*(10**4))/(K_f * K_u * B_ac * J * Frequency)
    return Area_product

def InputCurrent(Apparent_power, Input_voltage, Efficiency):
    # Calculation of input current
    InputCurrent = Apparent_power / ( Input_voltage * Efficiency * 0.01) 
    return InputCurrent
    
def bare_area(input_current, current_density):
    '''
        arguments: 
            current [amps], 
            current density [amp/]
        return variables: Bare area in sqmm
    '''
    a_wp = input_current / current_density
    return a_wp * 100

def findSWG(Bare_area):
    '''
        Bare area in sqmm
        return variables: 
            required swg dataframe, 
            diameter of insulated wire, 
            bare area of selected swg
    '''
    swg_data = data.swg_data
    # swg_data = pd.read_csv('../DATA/EMD - Sheet1.csv') # select the swg the data 
    higer_data = swg_data[Bare_area < swg_data['Normal Conductor Area mm²']]
    required_swg_result = higer_data.iloc[(higer_data['Normal Conductor Area mm²'] - Bare_area).abs().argsort()[:1]]
    diameter_of_insulated_wire = required_swg_result['Medium Covering Max']
    A_wp = required_swg_result['Normal Conductor Area mm²'].max() / 100 # cm^2
    return required_swg_result, diameter_of_insulated_wire.max(),  A_wp 
    
def calculate_stack(area_product, K_ratio):
    '''
    input arguments:
        area product at that time of lamination
        k-ratio from the lamination table data
    output variables:
        stack in mm
    '''
    stack_in_cm = area_product * 1000 / K_ratio
    stack = stack_in_cm * 10 # mm
    return stack 

def rounding_stack_as_multiple_of_five(stack):
    # for approximating for stack 
    if stack < 5:
        stack = 5.0
    elif stack%5 == 0:
        stack = stack
    elif stack%5 <= 2.5:
        stack = stack - stack % 5
    elif stack%5 > 2.5:
        stack = stack - stack % 5 + 5
    return stack

def Cu_Loss(Current, Resistance):
    '''
    Cu_loss = Current^2 * Resistance
    
    Input arguments: Current, Resistance
    
    Return variable: Copper loss
    '''
    return Current**2 * Resistance


def core_area(stack, tongue):
    '''
    Core Area = stack * tongue
    '''
    CoreArea = stack * tongue 
    return CoreArea




In [6]:
def Complete_Cost_function(Output_power, Efficiency, Frequency, K_f, K_u, B_ac, J,Output_voltage, Input_voltage):
   """
   This function derives from the user inputs like
   -> Frequeny, Input voltage, Output voltage, Regulation,
      K_f, K_u, B_ac, Current density, insulation thickness
      Rate of Cu, Rate of Iron

   Output consists of optimized cost with required parameters 
   for designing the transformer
   """

   # Adding data files into the funciton
   lamination_data = data.lamination_data
   swg_data = data.swg_data

   # Calculate the apparent power
   Apparent_power = Output_power * (1/(0.01*Efficiency) + 1)
   print('Apparent Power: ' + str(Apparent_power) + ' watts')

   # Calculate the area product
   Area_product = area_product(K_f, K_u, B_ac, J, Frequency, Apparent_power)
   print('Area product: ' + str(Area_product))

   # Calculate the input current
   Input_current = InputCurrent(Apparent_power=Apparent_power, Input_voltage= Input_voltage, Efficiency= Efficiency)
   print('Input current ' + str(Input_current) + ' [amps]')
   
   # Calculate Bare area of primary wire
   A_wp_in_sqmm = bare_area(Input_current, J) 
   print('Calculated Priamry Bare Area: ' + str(A_wp_in_sqmm)+ ' mm²')

   # Calculate swg for primary wire
   required_swg_primary, diameter_of_primary_wire_with_insulation, A_wp = findSWG(A_wp_in_sqmm)
   print('SWG: ' + str(required_swg_primary['SWG'].to_string(index=False)))
   print('Final Priamry Bare Area after selecting swg: ' + str(A_wp*100)+ ' mm²')
   print('Primary wire Diameter with enamel: ' + str(diameter_of_primary_wire_with_insulation.max()) + ' mm')

   # Find Secondary current
   Secondary_current = Output_power / Output_voltage

   # Calculate Bare Area for secondary wire in cm
   A_ws = bare_area(Secondary_current, J)
   print('Secondary Bare Area: ' + str(A_ws)+ ' mm²') 

   # Find SWG for secondary wire as well as Actual Bare are
   required_swg_secondary, diameter_of_wire_secondary_insulated , A_ws = findSWG(A_ws)
   print('SWG: ' + str(required_swg_secondary['SWG'].to_string(index=False)))
   print('Primary wire Diameter with enamel: ' + str(diameter_of_wire_secondary_insulated.max()) + ' mm')

   


   
# testing
if __name__ == '__main__':
   Complete_Cost_function(Output_power, Efficiency, Frequency, K_f, K_u, B_ac, J, Ouptut_voltage, Input_voltage)



Apparent Power: 513.1578947368421 watts
Area product: 153.6916255561272
Input current 4.69709743466217 [amps]
Calculated Priamry Bare Area: 1.8788389738648679 mm²
SWG: 16.0
Final Priamry Bare Area after selecting swg: 2.0765 mm²
Primary wire Diameter with enamel: 1.737 mm
Secondary Bare Area: 0.8695652173913043 mm²
SWG: 18.5
Primary wire Diameter with enamel: 1.216 mm


In [9]:

def find_cost(Stack, Tongue, ww, wl, A, B, C,): #InputCurrent, Bobbin_thickness, Resistivity_Conductor, Reisitivity_Core, Regulation, Temperature_rise_goal):
    stack_data = []

    Apparent_power = Output_power * (1/(0.01*Efficiency) + 1)
    Area_product = area_product(K_f, K_u, B_ac, J, Frequency, Apparent_power)
    Input_current = InputCurrent(Apparent_power=Apparent_power, Input_voltage= Input_voltage, Efficiency= Efficiency)
    A_wp_in_sqmm = bare_area(Input_current, J)
    A_wp = A_wp_in_sqmm / 100 # cm2
    required_swg_primary, diameter_of_primary_wire_with_insulation, A_wp = findSWG(A_wp_in_sqmm)
    Secondary_current = Output_power / Ouptut_voltage
    A_ws = bare_area(Secondary_current, J)
    A_ws = A_ws / 100 # cm2
    required_swg_secondary, diameter_of_wire_secondary_insulated , A_ws = findSWG(A_ws)
    # Input current
    Iin = Input_current
    
    # diameters of wires
    dw_p = diameter_of_primary_wire_with_insulation.max()
    dw_s = diameter_of_wire_secondary_insulated.max()

    # Calculate core Area
    A_c = core_area(Stack, Tongue)

    stack = Stack

    # Calculate number of primary turns
    Np = (Input_voltage* 10**4) / (K_f * B_ac * Frequency * A_c)
    
    Turns_per_layer_primary = wl / dw_p 
    Number_of_layer_primary = Np / Turns_per_layer_primary
    Built_primary = ( Bobbin_thickness + Number_of_layer_primary * dw_p ) 
    MTL_primary = 2 * (tongue + stack + 2 * Built_primary + 4 * Bobbin_thickness) / 10 # cm
    Length_primary = MTL_primary * Np  # cm 
    Primary_Resistence = Resistivity_Cu * Length_primary / A_wp 
    Primary_Cu_loss = Cu_Loss(Current=Iin, Resistance=Primary_Resistence)
    
    Ns = Np * Ouptut_voltage * (1 + Regulation / 100) / Input_voltage
    Turns_per_layer_secondary = wl / diameter_of_wire_secondary_insulated 
    Number_of_layer_secondary = Ns /  Turns_per_layer_secondary  
    Built_secondary = Number_of_layer_secondary * diameter_of_wire_secondary_insulated + insulation_thickness     # mm
    MTL_secondary = 2 * (tongue + stack + 4 * Built_primary + 2 * Built_secondary + 4 * Bobbin_thickness) / 10    # cm
    Length_secondary = MTL_secondary * Ns 
    Secondary_Resistance = Resistivity_Cu * Length_secondary / A_ws
    Secondary_Cu_loss = Cu_Loss(Current=Secondary_current, Resistance=Secondary_Resistance) 

    # Total built
    Total_Built = Built_primary + Built_secondary
    
    # Total Copper loss
    Total_Cu_loss = Primary_Cu_loss + Secondary_Cu_loss

    Core_loss_factor = 0.000557 * Frequency**a * B_ac**b 

    volume_of_core = stack * ( B * C - 2 * ww * wl ) # mm^3

    volume_of_core_in_cm3 = volume_of_core / 1000 # cm3 

    Density_of_core = 7.65 # g/cm^3

    Wt_of_core = Density_of_core * volume_of_core_in_cm3 * 0.97 # stacking factor 

    Wt_of_core_in_kg = Wt_of_core / 1000 # kilograms

    Core_loss = Core_loss_factor * Wt_of_core_in_kg

    Total_loss = Total_Cu_loss + Core_loss

    Total_surface_area = 2 * ( B * C + B * (stack + 2 * Total_Built) + C * (stack + 2 * Total_Built) ) / 100 #cm2

    Cu_surface_area = pi * Total_Built * wl + (pi/2)* Total_Built**2 - (pi/2)* tongue**2

    Core_surface_area = 2 * ( (B * C - 2 * ww * wl) + stack * (B+C) )

    psi_copper = abs(Total_Cu_loss / Cu_surface_area)
 
    psi_core = Core_loss / Core_surface_area 

    Temperature_rise_Cu = 450 * psi_copper**0.826

    Temperature_rise_Fe = 450 * psi_core**0.826
   
    psi = Total_loss / Total_surface_area 

    Temperature_rise = 450 * psi**0.826 

    # if Temperature_rise_Cu < Temperature_rise_goal and Temperature_rise_Fe < Temperature_rise_goal:
    if ww * 0.9 > Total_Built:
        Wt_of_Cu_in_kg = (Length_primary * required_swg_primary['Conductor Weight for 1000m/Kg'].max() + Length_secondary * required_swg_secondary['Conductor Weight for 1000m/Kg'].max() ) / 10**5
        Cost = Wt_of_core_in_kg * Rate_of_Fe + Wt_of_Cu_in_kg * Rate_of_Cu 
        table_data_stack_and_tongue = {
            'Temperature rise': Temperature_rise,
            'Area Product cm²': present_Area_product,
            'A': A,
            'B': B,
            'C': C,
            'Stack mm': Stack,
            'Tongue mm': Tongue,
            'wl mm': wl,
            'ww mm': ww,
            'N_p': Np,
            'N_s': Ns,
            'TPL P': Turns_per_layer_primary,
            'TPL S': Turns_per_layer_secondary,
            'Number of L P': Number_of_layer_primary,
            'Number of L S': Number_of_layer_secondary,
            'MTL P': MTL_primary,
            'MTL S': MTL_secondary,
            'Total Built mm': Total_Built,
            'psi cu': psi_copper,
            'psi fe': psi_core,
            'Total Cu loss': Total_Cu_loss,
            'Cu Surface area': Cu_surface_area,
            'Core Surface area': Core_surface_area,
            'Temperature rise Cu': Temperature_rise_Cu,
            'Temperature rise Fe': Temperature_rise_Fe,
            'Total Cu Cost': Wt_of_Cu_in_kg * Rate_of_Cu,
            'Total Fe Cost': Wt_of_core_in_kg * Rate_of_Fe,
            'Core Area A_c cm²': A_c ,
            'Cost': Cost,
        }
        stack_data.append(table_data_stack_and_tongue)
    
    return table_data_stack_and_tongue


In [10]:
lamination_data = data.lamination_data
swg_data = data.swg_data

stack_data = []

# Calculate the apparent power
Apparent_power = Output_power * (1/(0.01*Efficiency) + 1)
print('Apparent Power: ' + str(Apparent_power) + ' watts')

# Calculate the area product
Area_product = area_product(K_f, K_u, B_ac, J, Frequency, Apparent_power)
print('Area product: ' + str(Area_product))

# Calculate the input current
Input_current = InputCurrent(Apparent_power=Apparent_power, Input_voltage= Input_voltage, Efficiency= Efficiency)
print('Input current ' + str(Input_current) + ' [amps]')

# Calculate Bare area of primary wire
A_wp_in_sqmm = bare_area(Input_current, J) 
print('Calculated Priamry Bare Area: ' + str(A_wp_in_sqmm)+ ' mm²')

# Calculate swg for primary wire
required_swg_primary, diameter_of_primary_wire_with_insulation, A_wp = findSWG(A_wp_in_sqmm)
print('SWG: ' + str(required_swg_primary['SWG'].to_string(index=False)))
print('Final Priamry Bare Area after selecting swg: ' + str(A_wp*100)+ ' mm²')
print('Primary wire Diameter with enamel: ' + str(diameter_of_primary_wire_with_insulation.max()) + ' mm')

# Find Secondary current
Secondary_current = Output_power / Ouptut_voltage

# Calculate Bare Area for secondary wire in cm
A_ws = bare_area(Secondary_current, J)
print('Secondary Bare Area: ' + str(A_ws)+ ' mm²') 

# Find SWG for secondary wire as well as Actual Bare are
required_swg_secondary, diameter_of_wire_secondary_insulated , A_ws = findSWG(A_ws)
print('SWG: ' + str(required_swg_secondary['SWG'].to_string(index=False)))
print('Primary wire Diameter with enamel: ' + str(diameter_of_wire_secondary_insulated.max()) + ' mm')

for lamination in lamination_data['Type']:
    selected_lamination = lamination_data[lamination_data['Type'] == lamination ]
    for x in range(60, 141, 5):
        present_Area_product = x * Area_product * 0.01

        
        stack = calculate_stack(present_Area_product, selected_lamination['K-ratio'].max())

        if stack < 5 * selected_lamination['Tongue'].max():

            stack = rounding_stack_as_multiple_of_five(stack)# mm

            tongue = selected_lamination['Tongue'].max ()    # mm
            wl = selected_lamination['Winding-length'].max() # mm
            ww = selected_lamination['Winding-width'].max()  # mm 
            A = selected_lamination['A'].max()               # mm
            B = selected_lamination['B'].max()               # mm
            C = selected_lamination['C'].max()               # mm

            cost = find_cost(stack, tongue, wl, ww, A, B, C)
            
                   

Apparent Power: 513.1578947368421 watts
Area product: 153.6916255561272
Input current 4.69709743466217 [amps]
Calculated Priamry Bare Area: 1.8788389738648679 mm²
SWG: 16.0
Final Priamry Bare Area after selecting swg: 2.0765 mm²
Primary wire Diameter with enamel: 1.737 mm
Secondary Bare Area: 0.8695652173913043 mm²
SWG: 18.5
Primary wire Diameter with enamel: 1.216 mm
